In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Carregar os dados de geração e clima
generation_data = pd.read_csv('Plant_2_Generation_Dataset.csv')
weather_data = pd.read_csv('Plant_2_Weather_Sensor_Data.csv')

# Converter 'DATE_TIME' para datetime
generation_data['DATE_TIME'] = pd.to_datetime(generation_data['DATE_TIME'])
weather_data['DATE_TIME'] = pd.to_datetime(weather_data['DATE_TIME'])

# Mesclar os dados com base na coluna 'DATE_TIME'
df_solar = pd.merge(
    generation_data.drop(columns=['PLANT_ID']),
    weather_data.drop(columns=['PLANT_ID', 'SOURCE_KEY']),
    on='DATE_TIME'
)

# Extrair a data e a hora
df_solar['DATE'] = df_solar['DATE_TIME'].dt.date
df_solar['HOUR'] = df_solar['DATE_TIME'].dt.hour

# Agrupar por data e hora e calcular a média da irradiação
hourly_irradiation = df_solar.groupby(['DATE', 'HOUR'])['IRRADIATION'].mean().reset_index()

# Criar uma lista de dicionários com hora e índice de irradiação
irradiation_dicts = []
dates = hourly_irradiation['DATE'].unique()

for date in dates:
    day_data = hourly_irradiation[hourly_irradiation['DATE'] == date]
    irradiation_dict = dict(zip(day_data['HOUR'], day_data['IRRADIATION']))
    irradiation_dicts.append({'DATE': date, 'IRRADIATION_BY_HOUR': irradiation_dict})

# Preparar os dados para o LSTM
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Converter a lista de dicionários em um DataFrame
irradiation_df = pd.DataFrame(irradiation_dicts)

# Expandir o dicionário de irradiação em colunas
irradiation_expanded = irradiation_df['IRRADIATION_BY_HOUR'].apply(pd.Series).fillna(0)
irradiation_expanded['DATE'] = irradiation_df['DATE']

# Obter o rendimento total diário
daily_total_yield = df_solar.groupby('DATE')['TOTAL_YIELD'].max().reset_index()
daily_total_yield.rename(columns={'TOTAL_YIELD': 'TOTAL_DAILY_YIELD'}, inplace=True)

# Combinar os dados
final_data = pd.merge(irradiation_expanded, daily_total_yield, on='DATE')

# Preparar os dados para treinamento
X = final_data.drop(columns=['DATE', 'TOTAL_DAILY_YIELD']).values
y = final_data['TOTAL_DAILY_YIELD'].values

# Escalonar os dados
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Remodelar os dados para [amostras, timesteps, features]
X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Dividir os dados em conjuntos de treinamento e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Treinar o modelo
model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=0)

# Avaliar o modelo
y_pred = model.predict(X_test)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"R² do modelo: {r2:.2f}")

# Salvar o modelo
model.save('modelo_solar_lstm.h5')

ModuleNotFoundError: No module named 'distutils'